## 4.4 VGG16

Data Preprocess of VGG16

In [ ]:
import os
import numpy as np
from os import listdir
from imageio import imread
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.utils.image_utils import img_to_array
import keras
import PIL
import matplotlib.pyplot as plt

In [ ]:
# Settings 
num_classes = 10
test_size = 0.2

Read Image and Convert to 3D Array

In [ ]:
def get_img(data_path): 
  ## Getting image array from path: 
  img = PIL.Image.open(data_path)
  img = img.convert("L")
  img = img_to_array(img)
  img = np.resize(img, (100, 100, 3))
  return img

Get dataset from picture and then split to train and test set

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/Dataset"

## Getting all data from data path
labels = sorted(listdir(dataset_path))
X = []
Y = []
for i, label in enumerate(labels): 
  data_path = dataset_path + "/" + label
  
  for data in listdir(data_path): 
    img = get_img(data_path + "/" + data)
    X.append(img)
    Y.append(i)
## create dataset 
X = 1 - np.array(X).astype("float32") /255
Y = np.array(Y).astype("float32")
Y = to_categorical(Y, num_classes)

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=test_size, random_state = 42)
print(X.shape)
print(X_test.shape)
print(Y.shape)
print(Y_test.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1649, 100, 100, 3)
(413, 100, 100, 3)
(1649, 10)
(413, 10)


In [ ]:
import tensorflow as tf
from numpy.random import seed
seed(1)
tf.random.set_seed(123)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import sklearn as sk
import time
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras import initializers
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import math 
from keras import applications

In [ ]:
img_height = 100
img_width = 100

# VGG16

Learning rate has been adjusted between le-3, le-4, le-5, le-6, the result is le-5 can provide the best performance. Drop out rate of 0.3, 0.4, 0.5 has been tried and ultimately 0.4 has a relativley good performance. Initially epoch has been set to 10 but the performance was not pretty well. By increasing to 50, the model has been trained into the accuracy of 98%
GlobalAveragePooling2D()for creating feature map for each category of the model and unfreezing the base model and retrain the whole model for fine-tuning has been applied in all transfer learning model.

In [ ]:
from tensorflow.keras.applications import VGG16
#base_model = VGG16(include_top = False, weights = 'imagenet',input_shape=(100,100,3))
base_model = keras.applications.VGG16(
    weights='imagenet', 
    input_shape=(img_height, img_width, 3),
    include_top=False)
#base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(img_height, img_width, 3))

In [ ]:
x=tf.keras.applications.vgg16.preprocess_input(
    inputs, data_format=None
)

In [ ]:
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.4)(x)
outputs = keras.layers.Dense(10)(x)

model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 100, 100, 3)]     0         
                                                                 
 tf.__operators__.getitem_4   (None, 100, 100, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_4 (TFOpLambd  (None, 100, 100, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 global_average_pooling2d_4   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                           

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(X, Y, epochs=3, validation_data=(X_test,Y_test))

Epoch 1/3
52/52 [==============================] - 4s 58ms/step - loss: 4.3663 - accuracy: 0.1049 - val_loss: 2.3922 - val_accuracy: 0.0969
Epoch 2/3
52/52 [==============================] - 2s 48ms/step - loss: 2.8542 - accuracy: 0.1104 - val_loss: 2.3356 - val_accuracy: 0.0847
Epoch 3/3
52/52 [==============================] - 3s 48ms/step - loss: 2.5525 - accuracy: 0.1013 - val_loss: 2.3267 - val_accuracy: 0.0847


In [ ]:
# fine-tuning
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

epochs = 50
model.fit(X, Y, epochs=epochs, validation_data=(X_test,Y_test))

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 100, 100, 3)]     0         
                                                                 
 tf.__operators__.getitem_4   (None, 100, 100, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_4 (TFOpLambd  (None, 100, 100, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 global_average_pooling2d_4   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                           

Accuracy: 98.73%

By comparing model of transfered learning model with 
Based on the results from model we built from scratch, the accuracy of fully connected model is 71%, accuracy of CNN model is 80.34%. All the five models of transfer learning have a better performance than those two since transfer learning will include the saving of resources and improve efficiety when training new models with complex layers. 